In [1]:
import pandas as pd


In [3]:

# URL del archivo CSV en GitHub
url = 'https://raw.githubusercontent.com/Yeider07/Gobierno-Dacto-Act_1/refs/heads/main/DevOps_0/Licencias_Terrazas_Integradas.csv'

datos = pd.read_csv(url)
print("Datos cargados exitosamente:")
print(datos.head())  # Muestra las primeras filas del DataFrame


Datos cargados exitosamente:
   Unnamed: 0_x  id_terraza   id_local  id_distrito_local_x  \
0             0           7  280067128                   20   
1             1          33  270403150                    4   
2             2          34  270078567                    4   
3             2          34  270078567                    4   
4             3          36   40002021                    4   

  desc_distrito_local_x  id_barrio_local_x   desc_barrio_local_x  \
0  SAN BLAS-CANILLEJAS                2005  ROSAS                  
1  SALAMANCA                           404  GUINDALERA             
2  SALAMANCA                           404  GUINDALERA             
3  SALAMANCA                           404  GUINDALERA             
4  SALAMANCA                           404  GUINDALERA             

   id_ndp_edificio_x  id_vial_edificio_x     clase_vial_edificio_x  ...  \
0           20157611                2443  CALLE                     ...   
1           11018870             

In [6]:
################# elimina los registros que contengan valores nulos en más del 50% de sus columnas #################################

# Si una fila tiene menos del 50% de valores no nulos, se eliminará
umbral = len(datos.columns) * 0.5

# Eliminar filas que tienen más del 50% de valores nulos
filtrado_df = datos.dropna(thresh=umbral + 1)

# Calcular el número de registros eliminados
registros_eliminados = len(datos) - len(filtrado_df)

# Mostrar cuántos valores nulos tiene cada columna
print(datos.isnull().sum())
# número de registros eliminados
print(f"\nNúmero de registros eliminados: {registros_eliminados}")

Unnamed: 0_x                    0
id_terraza                      0
id_local                        0
id_distrito_local_x             0
desc_distrito_local_x           0
                               ..
id_tipo_licencia                0
desc_tipo_licencia              0
id_tipo_situacion_licencia      0
desc_tipo_situacion_licencia    0
Fecha_Dec_Lic                   0
Length: 93, dtype: int64

Número de registros eliminados: 0


In [12]:
########################## Detección y Eliminación de Duplicados ##############################################################

# Número de registros antes de eliminar duplicados
Contar_Original = len(datos)
print(f'Cantidad de registros: {Contar_Original}')

# Identificar y eliminar duplicados basados en las columnas 'id_local' y 'ref_licencia'
Licencias_SinDuplicados = datos.drop_duplicates(subset=['id_local', 'ref_licencia'])

# Número de registros después de eliminar duplicados
Contar = len(Licencias_SinDuplicados)

# Imprimir la cantidad de registros eliminados
print(f"Registros eliminados: {Contar_Original - Contar}")


Cantidad de registros: 13510
Registros eliminados: 48


In [28]:
################# Transformar columnas al formato correcto ########################################

datos['hora_ini_LJ_es'] = pd.to_datetime(datos['hora_ini_LJ_es'], errors='coerce')
datos['hora_fin_LJ_es'] = pd.to_datetime(datos['hora_fin_LJ_es'], errors='coerce')
datos['hora_ini_LJ_ra'] = pd.to_datetime(datos['hora_ini_LJ_ra'], errors='coerce')
datos['hora_fin_LJ_ra'] = pd.to_datetime(datos['hora_fin_LJ_ra'], errors='coerce')
datos['hora_ini_VS_es'] = pd.to_datetime(datos['hora_ini_VS_es'], errors='coerce')
datos['hora_fin_VS_es'] = pd.to_datetime(datos['hora_fin_VS_es'], errors='coerce')
datos['hora_ini_VS_ra'] = pd.to_datetime(datos['hora_ini_VS_ra'], errors='coerce')
datos['hora_fin_VS_ra'] = pd.to_datetime(datos['hora_fin_VS_ra'], errors='coerce')

# Convertir la columna 'sillas_ra' a tipo numérico
datos['sillas_ra'] = pd.to_numeric(datos['sillas_ra'], errors='coerce')

# Imprimir el tipo de dato de cada una de las columnas
print("Tipos de datos de cada columna:")
print(datos.dtypes)

# Crear una nueva columna que calcule el ratio entre "Superficie_TO" y "id_terraza"
datos['Superficie_TO'] =  datos['Superficie_ES'] + datos['Superficie_RA']
datos['Ratio'] = datos['Superficie_TO'] / datos['id_terraza']

Tipos de datos de cada columna:
Unnamed: 0_x                     int64
id_terraza                       int64
id_local                         int64
id_distrito_local_x              int64
desc_distrito_local_x           object
                                 ...  
id_tipo_licencia                 int64
desc_tipo_licencia              object
id_tipo_situacion_licencia       int64
desc_tipo_situacion_licencia    object
Fecha_Dec_Lic                   object
Length: 93, dtype: object


In [34]:
# Identificar columnas que terminan en '_x'
columnas_eliminadas = datos.columns[datos.columns.str.endswith('_x')]

# Eliminar columnas que terminan en '_x'
datos = datos.loc[:, ~datos.columns.str.endswith('_x')]

# Imprimir las columnas eliminadas
print("Columnas eliminadas:")
print(columnas_eliminadas)

# Eliminar la columna que tiene un nombre en blanco
datos = datos.loc[:, datos.columns != '']

Columnas eliminadas:
Index([], dtype='object')


In [39]:
################################# Guardar los datos ##########################
datos.to_csv('DataTerrazas.csv')

In [37]:
import pyodbc
import os
conexion = pyodbc.connect('DRIVER={SQL SERVER};'
                           'SERVER=DESKTOP-FE1O4UB;'
                           'database=Gob_Datos_Act1_inmon;'
                           'Trusted_Connection=yes;')

Cursor = conexion.cursor()

# Query para ejecutar el BULK INSERT
query = """
BULK INSERT [Gob_Datos_Devops_Act1].[dbo].[DataTerrazas]
FROM 'C:\\Users\\Yeider\\Desktop\\Master Visual Analitics And Big Data\\Gobierno del dato y toma de decisiones\\Actividades\\Actividad 1\\DevOps\\DataTerrazas.csv'
WITH (
    FIELDTERMINATOR = ',',  
    FIRSTROW = 2,            
    FORMAT = 'CSV',          
    FIELDQUOTE = '"',        
    TABLOCK,                 
    ORDER (
[id_terraza],
[id_local],
[Cod_Postal],
[id_periodo_terraza],
[desc_periodo_terraza],
[id_situacion_terraza],
[desc_situacion_terraza],
[Superficie_ES],
[Superficie_RA],
[Fecha_confir_ult_decreto_resol],
[id_ndp_terraza],
[ID_VIAL],
[DESC_CLASE],
[DESC_NOMBRE],
[num_terraza],
[cal_terraza],
[desc_ubicacion_terraza],
[hora_ini_LJ_es],
[hora_fin_LJ_es],
[hora_ini_LJ_ra],
[hora_fin_LJ_ra],
[hora_ini_VS_es],
[hora_fin_VS_es],
[hora_ini_VS_ra],
[hora_fin_VS_ra],
[mesas_aux_es],
[mesas_aux_ra],
[mesas_es],
[mesas_ra],
[sillas_es],
[sillas_ra],
[Superficie_TO],
[Ratio],
[Unnamed: 0_y],
[id_distrito_local_y],
[desc_distrito_local_y],
[id_barrio_local_y],
[desc_barrio_local_y],
[cod_barrio_local],
[id_seccion_censal_local],
[desc_seccion_censal_local],
[coordenada_x_local_y],
[coordenada_y_local_y],
[id_tipo_acceso_local_y],
[desc_tipo_acceso_local_y],
[id_situacion_local_y],
[desc_situacion_local_y],
[id_ndp_edificio_y],
[id_vial_edificio_y],
[clase_vial_edificio_y],
[desc_vial_edificio_y],
[num_edificio_y],
[cal_edificio],
[secuencial_local_PC_y],
[id_ndp_acceso],
[id_vial_acceso],
[clase_vial_acceso],
[desc_vial_acceso],
[nom_acceso],
[num_acceso],
[cal_acceso],
[id_agrupacion],
[nombre_agrupacion],
[id_tipo_agrup],
[desc_tipo_agrup],
[id_planta_agrupado_y],
[rotulo_y],
[ref_licencia],
[id_tipo_licencia],
[desc_tipo_licencia],
[id_tipo_situacion_licencia],
[desc_tipo_situacion_licencia],
[Fecha_Dec_Lic]
    )  
);
"""

print('Se guardo la informacion en la Base de datos')

In [ ]:
# Crear Datasets filtrados para tablas y cargar a sql server

import pyodbc
import os

conexion = pyodbc.connect('DRIVER={SQL SERVER};'
                           'SERVER=DESKTOP-FE1O4UB;'
                           'database=Gob_Datos_Act1_inmon;'
                           'Trusted_Connection=yes;')
Cursor = conexion.cursor()

#Filtrar y guardar 
Colum_Terrazas = [
'id_terraza',
'id_local',
'id_periodo_terraza',
'id_situacion_terraza',
'Superficie_ES',
'Superficie_RA',
'Superficie_TO',
]
Terrazas = datos[Colum_Terrazas]
Terrazas.to_csv('Terrazas.csv')

#Importar Datos SQL Server
query = """
BULK INSERT [Gob_Datos_Devops_Act1].[dbo].[DataTerrazas]
FROM 'C:\\Users\\Yeider\\Desktop\\Master Visual Analitics And Big Data\\Gobierno del dato y toma de decisiones\\Actividades\\Actividad 1\\DevOps\\DataTerrazas.csv'
WITH (
    FIELDTERMINATOR = ',',  
    FIRSTROW = 2,            
    FORMAT = 'CSV',          
    FIELDQUOTE = '"',        
    TABLOCK,                 
    ORDER (
[id_terraza],
[id_local],
[id_periodo_terraza],
[id_situacion_terraza],
[Superficie_ES],
[Superficie_RA],
[Superficie_TO],
    )  
);
"""
print('Se guardo la informacion en la Base de datos')

Colum_Locales = [
'id_local',
'id_distrito_local_x',
'id_distrito_local_y', 
'id_barrio_local_x',
'id_barrio_local_y',
'id_tipo_acceso_local_x',
'id_tipo_acceso_local_y', 
'id_situacion_local_x',
'id_situacion_local_y',
'coordenada_x_local_x',
'coordenada_x_local_y',
'coordenada_y_local_x',
'coordenada_y_local_y',
'rotulo_y',
'rotulo_x',
'id_tipo_licencia'
]
Locales = datos[Colum_Locales]
Locales.to_csv('locales.csv')

#Importar Datos SQL Server
query = """
BULK INSERT [Gob_Datos_Devops_Act1].[dbo].[DataTerrazas]
FROM 'C:\\Users\\Yeider\\Desktop\\Master Visual Analitics And Big Data\\Gobierno del dato y toma de decisiones\\Actividades\\Actividad 1\\DevOps\\DataTerrazas.csv'
WITH (
    FIELDTERMINATOR = ',',  
    FIRSTROW = 2,            
    FORMAT = 'CSV',          
    FIELDQUOTE = '"',        
    TABLOCK,                 
    ORDER (
[id_local],
[id_distrito_local_x],
[id_distrito_local_y], 
[id_barrio_local_x],
[id_barrio_local_y],
[id_tipo_acceso_local_x],
[id_tipo_acceso_local_y], 
[id_situacion_local_x],
[id_situacion_local_y],
[coordenada_x_local_x],
[coordenada_x_local_y],
[coordenada_y_local_x],
[coordenada_y_local_y],
[rotulo_y],
[rotulo_x],
[id_tipo_licencia]
    )  
);
"""
print('Se guardo la informacion en la Base de datos')


Colum_Ubicaciones = [
'id_distrito_local_x',
'id_distrito_local_y',
'desc_distrito_local_x',
'desc_distrito_local_y'
]
ubicaciones = datos[Colum_Ubicaciones]
ubicaciones.to_csv('ubicaciones.csv')

#Importar Datos SQL Server
query = """
BULK INSERT [Gob_Datos_Devops_Act1].[dbo].[DataTerrazas]
FROM 'C:\\Users\\Yeider\\Desktop\\Master Visual Analitics And Big Data\\Gobierno del dato y toma de decisiones\\Actividades\\Actividad 1\\DevOps\\DataTerrazas.csv'
WITH (
    FIELDTERMINATOR = ',',  
    FIRSTROW = 2,            
    FORMAT = 'CSV',          
    FIELDQUOTE = '"',        
    TABLOCK,                 
    ORDER (
[id_distrito_local_x],
[id_distrito_local_y],
[desc_distrito_local_x],
[desc_distrito_local_y]
    )  
);
"""
print('Se guardo la informacion en la Base de datos')

Colum_Accesos_Locales = [
'id_tipo_acceso_local_x',
'id_tipo_acceso_local_y',
'desc_tipo_acceso_local_x',
'desc_tipo_acceso_local_y'
]
Accesos_Locales = datos[Colum_Accesos_Locales]
Accesos_Locales.to_csv('Accesos_Locales.csv')


Colum_Situacion_Local = [
'id_situacion_local_x',
'id_situacion_local_y',
'desc_situacion_local_x',
'desc_situacion_local_y'
]
Situacion_Local = datos[Colum_Situacion_Local]
Situacion_Local.to_csv('Situacion_Local.csv')


Colum_Licencias = [
#'id_licencia',
'id_tipo_licencia',
'desc_tipo_licencia',
'Fecha_Dec_Lic']
Licencias = datos[Colum_Licencias]
Licencias.to_csv('Licencias.csv')


Colum_Situacion_Licencia = [
'id_tipo_situacion_licencia',
'desc_tipo_situacion_licencia'
]
Situacion_Licencia = datos[Colum_Situacion_Licencia]
Situacion_Licencia.to_csv('Situacion_Licencia.csv')


Colum_Periodos = [
'id_periodo_terraza',
'desc_periodo_terraza'
]
Periodos = datos[Colum_Periodos]
Periodos.to_csv('Periodos.csv')